<a href="https://colab.research.google.com/github/bawantha/ride_fare/blob/master/RideFarePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

import os

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare
count,1.717600e+04,16974.000000,16974.000000,16974.000000,16974.000000,16974.000000,17176.000000,17176.000000,17176.000000,17176.000000,17039.000000
mean,2.014549e+08,13.719651,1702.858077,629.074231,32.057666,112.466832,6.965499,79.998389,6.968587,79.997534,312.655466
std,7.456830e+06,98.975052,8120.160818,8063.260669,465.483573,442.054812,0.210760,0.243384,0.378669,0.359648,605.300565
min,1.891236e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.937600,79.819100,5.941860,45.307700,0.000000
25%,1.947619e+08,10.500000,548.000000,33.000000,0.000000,26.000000,6.865077,79.870400,6.865625,79.870100,125.515000
50%,2.013582e+08,10.500000,1009.500000,123.000000,0.116600,73.000000,6.914105,79.898600,6.916160,79.899100,211.180000
75%,2.083081e+08,10.500000,1803.000000,351.000000,14.341800,144.000000,7.043057,79.956600,7.047195,79.960225,366.465000
max,2.138173e+08,12795.030000,453872.000000,453650.000000,26198.601400,51288.000000,8.051010,81.009600,48.132000,81.009700,26945.990000


In [ ]:
test.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare
count,8.576000e+03,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000
mean,2.180067e+08,11.088095,1541.590485,508.502099,24.671655,119.863106,6.969246,80.008728,6.970008,80.010145,293.278057
std,2.789252e+06,5.359602,5952.458574,5784.005855,333.515641,793.233190,0.217809,0.251873,0.218826,0.252229,436.730282
min,2.132846e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.943130,79.817700,5.936600,79.818000,0.000000
25%,2.156247e+08,10.500000,540.000000,30.000000,0.000000,26.000000,6.864810,79.871600,6.864950,79.871300,126.077500
50%,2.179006e+08,10.500000,982.500000,116.000000,0.000000,72.000000,6.913605,79.900500,6.915460,79.902250,203.950000
75%,2.203904e+08,10.500000,1723.000000,336.250000,11.744200,138.000000,7.068875,79.963225,7.073505,79.967825,344.075000
max,2.228607e+08,160.500000,447230.000000,446934.000000,25760.361800,55571.000000,8.425010,80.776400,8.421730,80.805500,26844.330000


In [ ]:
train.isnull().sum().sort_values(ascending=False)


meter_waiting_till_pickup    202
meter_waiting_fare           202
meter_waiting                202
duration                     202
additional_fare              202
fare                         137
label                          0
drop_lon                       0
drop_lat                       0
pick_lon                       0
pick_lat                       0
drop_time                      0
pickup_time                    0
tripid                         0
dtype: int64

In [ ]:

Q1 = train.quantile(0.05)
Q3 = train.quantile(0.95)
IQR = Q3 - Q1
train = train[~((train < (Q1 - 1.5 * IQR)) |(train > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
test.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare
count,8.576000e+03,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000
mean,2.180067e+08,11.088095,1541.590485,508.502099,24.671655,119.863106,6.969246,80.008728,6.970008,80.010145,293.278057
std,2.789252e+06,5.359602,5952.458574,5784.005855,333.515641,793.233190,0.217809,0.251873,0.218826,0.252229,436.730282
min,2.132846e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.943130,79.817700,5.936600,79.818000,0.000000
25%,2.156247e+08,10.500000,540.000000,30.000000,0.000000,26.000000,6.864810,79.871600,6.864950,79.871300,126.077500
50%,2.179006e+08,10.500000,982.500000,116.000000,0.000000,72.000000,6.913605,79.900500,6.915460,79.902250,203.950000
75%,2.203904e+08,10.500000,1723.000000,336.250000,11.744200,138.000000,7.068875,79.963225,7.073505,79.967825,344.075000
max,2.228607e+08,160.500000,447230.000000,446934.000000,25760.361800,55571.000000,8.425010,80.776400,8.421730,80.805500,26844.330000


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8576 entries, 0 to 8575
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     8576 non-null   int64  
 1   additional_fare            8576 non-null   float64
 2   duration                   8576 non-null   int64  
 3   meter_waiting              8576 non-null   int64  
 4   meter_waiting_fare         8576 non-null   float64
 5   meter_waiting_till_pickup  8576 non-null   int64  
 6   pickup_time                8576 non-null   object 
 7   drop_time                  8576 non-null   object 
 8   pick_lat                   8576 non-null   float64
 9   pick_lon                   8576 non-null   float64
 10  drop_lat                   8576 non-null   float64
 11  drop_lon                   8576 non-null   float64
 12  fare                       8576 non-null   float64
dtypes: float64(7), int64(4), object(2)
memory usage:

In [ ]:
test.isnull().sum().sort_values(ascending=False)

fare                         0
drop_lon                     0
drop_lat                     0
pick_lon                     0
pick_lat                     0
drop_time                    0
pickup_time                  0
meter_waiting_till_pickup    0
meter_waiting_fare           0
meter_waiting                0
duration                     0
additional_fare              0
tripid                       0
dtype: int64

In [ ]:
def haversine_distance(lat1, long1, lat2, long2):
    data = [train, test]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

In [ ]:
haversine_distance('pick_lat', 'pick_lon', 'drop_lat', 'drop_lon')

0        6.705702
1       41.558513
2        5.916678
3        3.301761
4        2.588542
          ...    
8571     3.934272
8572     7.517433
8573     2.057225
8574     3.900888
8575     5.435270
Length: 8576, dtype: float64

In [ ]:
train['H_Distance'].head(10)

0     5.092770
1     3.168058
2     6.305395
3     0.861946
4     8.147782
5    24.207039
6     4.777624
7     5.322544
8     1.035302
9     2.930715
Name: H_Distance, dtype: float64

In [ ]:
test['H_Distance'].head(10)

0     6.705702
1    41.558513
2     5.916678
3     3.301761
4     2.588542
5    17.247478
6     3.132721
7    11.556896
8     6.458780
9     4.218669
Name: H_Distance, dtype: float64

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8576 entries, 0 to 8575
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     8576 non-null   int64  
 1   additional_fare            8576 non-null   float64
 2   duration                   8576 non-null   int64  
 3   meter_waiting              8576 non-null   int64  
 4   meter_waiting_fare         8576 non-null   float64
 5   meter_waiting_till_pickup  8576 non-null   int64  
 6   pickup_time                8576 non-null   object 
 7   drop_time                  8576 non-null   object 
 8   pick_lat                   8576 non-null   float64
 9   pick_lon                   8576 non-null   float64
 10  drop_lat                   8576 non-null   float64
 11  drop_lon                   8576 non-null   float64
 12  fare                       8576 non-null   float64
 13  H_Distance                 8576 non-null   float

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16676 entries, 0 to 17175
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     16676 non-null  int64  
 1   additional_fare            16474 non-null  float64
 2   duration                   16474 non-null  float64
 3   meter_waiting              16474 non-null  float64
 4   meter_waiting_fare         16474 non-null  float64
 5   meter_waiting_till_pickup  16474 non-null  float64
 6   pickup_time                16676 non-null  object 
 7   drop_time                  16676 non-null  object 
 8   pick_lat                   16676 non-null  float64
 9   pick_lon                   16676 non-null  float64
 10  drop_lat                   16676 non-null  float64
 11  drop_lon                   16676 non-null  float64
 12  fare                       16539 non-null  float64
 13  label                      16676 non-null  obj

In [ ]:
train=train.fillna(train.mean())

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16676 entries, 0 to 17175
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     16676 non-null  int64  
 1   additional_fare            16676 non-null  float64
 2   duration                   16676 non-null  float64
 3   meter_waiting              16676 non-null  float64
 4   meter_waiting_fare         16676 non-null  float64
 5   meter_waiting_till_pickup  16676 non-null  float64
 6   pickup_time                16676 non-null  object 
 7   drop_time                  16676 non-null  object 
 8   pick_lat                   16676 non-null  float64
 9   pick_lon                   16676 non-null  float64
 10  drop_lat                   16676 non-null  float64
 11  drop_lon                   16676 non-null  float64
 12  fare                       16676 non-null  float64
 13  label                      16676 non-null  obj

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8576 entries, 0 to 8575
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     8576 non-null   int64  
 1   additional_fare            8576 non-null   float64
 2   duration                   8576 non-null   int64  
 3   meter_waiting              8576 non-null   int64  
 4   meter_waiting_fare         8576 non-null   float64
 5   meter_waiting_till_pickup  8576 non-null   int64  
 6   pickup_time                8576 non-null   object 
 7   drop_time                  8576 non-null   object 
 8   pick_lat                   8576 non-null   float64
 9   pick_lon                   8576 non-null   float64
 10  drop_lat                   8576 non-null   float64
 11  drop_lon                   8576 non-null   float64
 12  fare                       8576 non-null   float64
 13  H_Distance                 8576 non-null   float

In [ ]:
train.isnull().sum().sort_values(ascending=False)

H_Distance                   0
label                        0
fare                         0
drop_lon                     0
drop_lat                     0
pick_lon                     0
pick_lat                     0
drop_time                    0
pickup_time                  0
meter_waiting_till_pickup    0
meter_waiting_fare           0
meter_waiting                0
duration                     0
additional_fare              0
tripid                       0
dtype: int64

In [ ]:
test['pickup_time']  = pd.to_datetime(test['pickup_time'])
test['drop_time']  = pd.to_datetime(test['pickup_time'])
train['pickup_time']  = pd.to_datetime(train['pickup_time'])
train['drop_time']  = pd.to_datetime(train['pickup_time'])

In [ ]:
data = [train,test]
for i in data:
    i['drop_time_Year'] = i['drop_time'].dt.year
    i['drop_time_Month'] = i['drop_time'].dt.month
    i['drop_time_Date'] = i['drop_time'].dt.day
    i['drop_time_Day of Week'] = i['drop_time'].dt.dayofweek
    i['drop_time_Hour'] = i['drop_time'].dt.hour

In [ ]:
data = [train,test]
for i in data:
    i['pickup_time_Year'] = i['pickup_time'].dt.year
    i['pickup_time_Month'] = i['pickup_time'].dt.month
    i['pickup_time_Date'] = i['pickup_time'].dt.day
    i['pickup_time_Day of Week'] = i['pickup_time'].dt.dayofweek
    i['pickup_time_Hour'] = i['pickup_time'].dt.hour

In [ ]:
d=['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon']
data=[test,train]
for i in data:
  i=i.drop(d,axis=1)

In [ ]:
train.loc[train['label'] =="correct", 'label'] = 1
train.loc[train['label'] =="incorrect", 'label'] = 0

In [ ]:
pip install catboost

     |████████████████████████████████| 64.8MB 48kB/s 


In [ ]:
train.shape

(16676, 25)

In [ ]:
test.shape

(8576, 24)

In [ ]:
from catboost import CatBoostClassifier


In [ ]:
x_train=train.loc[:, train.columns != 'label']

In [ ]:
y_train=train.loc[:,'label']

In [ ]:
x_train.shape

(16676, 24)

In [ ]:
y_train.shape

(16676,)

In [ ]:
y_train=y_train.astype(str).astype(int)

In [ ]:
model_cb = CatBoostClassifier(task_type='GPU', iterations=100000, 
                              random_state = 2021,)

In [ ]:
model_cb.fit(x_train, y_train)

Streaming output truncated to the last 5000 lines.
95001:	learn: 0.0924317	total: 1h 13m 33s	remaining: 3m 52s
95002:	learn: 0.0924317	total: 1h 13m 33s	remaining: 3m 52s
95003:	learn: 0.0924317	total: 1h 13m 33s	remaining: 3m 52s
95004:	learn: 0.0924307	total: 1h 13m 33s	remaining: 3m 52s
95005:	learn: 0.0924307	total: 1h 13m 33s	remaining: 3m 51s
95006:	learn: 0.0924305	total: 1h 13m 33s	remaining: 3m 51s
95007:	learn: 0.0924298	total: 1h 13m 33s	remaining: 3m 51s
95008:	learn: 0.0924296	total: 1h 13m 33s	remaining: 3m 51s
95009:	learn: 0.0924291	total: 1h 13m 33s	remaining: 3m 51s
95010:	learn: 0.0924291	total: 1h 13m 33s	remaining: 3m 51s
95011:	learn: 0.0924291	total: 1h 13m 33s	remaining: 3m 51s
95012:	learn: 0.0924288	total: 1h 13m 33s	remaining: 3m 51s
95013:	learn: 0.0924286	total: 1h 13m 33s	remaining: 3m 51s
95014:	learn: 0.0924286	total: 1h 13m 33s	remaining: 3m 51s
95015:	learn: 0.0924285	total: 1h 13m 33s	remaining: 3m 51s
95016:	learn: 0.0924285	total: 1h 13m 33s	remaini

In [ ]:
predictions = model_cb.predict(test)

In [ ]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
submission=pd.DataFrame({'tripid':test['tripid'],'prediction':predictions[:]})

In [ ]:
from google.colab import files

In [ ]:
submission.to_csv('submission.csv',index = False)

In [ ]:
files.download('submission.csv')